In [3]:
import pickle

In [25]:
import numpy as np
import pandas as pd

# sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import NMF
from sklearn.metrics.pairwise import cosine_similarity

import nltk
import spacy

In [4]:
with open('/Users/robertpagano/metis_data/project_4/text_dataframes/clustering/df_lsa_model_20.pickle', 'rb') as f:
    df_lsa_model_20 = pickle.load(f)

In [12]:
def pre_process2(text):
    
    # lowercase
    text=text.lower()
    
    #remove tags
    text=re.sub("&lt;/?.*?&gt;"," &lt;&gt; ",text)
    
    # remove special characters and digits
    text=re.sub("(\\d|\\W)+"," ",text)
    
    #lemmatize with Spacy
    doc = nlp(text)
    text = " ".join([token.lemma_ for token in doc])
    
    return text

In [33]:
vectorizer = TfidfVectorizer(stop_words='english', token_pattern="\\b[a-z][a-z]+\\b", ngram_range=(1,2), sublinear_tf=True)


In [34]:
bag_of_words_4 = vectorizer.fit_transform(X_blurb_2)

NameError: name 'X_blurb_2' is not defined

In [16]:
lsa = TruncatedSVD(20)

In [19]:
df_lsa_model_20.head()

,topic_id,topic_blurb,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,...,topic_10,topic_11,topic_12,topic_13,topic_14,topic_15,topic_16,topic_17,topic_18,topic_19
0,269726791,An artistic professional pack of playing cards...,0.118765,0.182466,-0.037672,0.039376,-0.048429,0.045518,0.009710,0.012686,...,-0.014650,0.027404,0.020749,0.015065,0.015533,-0.002469,0.010655,0.004937,-0.011222,0.014631
1,1697621882,Make it rain pancakes in FAT STACKS. \nCreate ...,0.052765,-0.033622,-0.045537,-0.032304,-0.015172,0.015924,0.008454,-0.017053,...,0.026820,0.029376,-0.045896,0.045269,0.014085,0.038517,-0.002483,-0.040691,-0.013559,0.030872
2,2046938895,"Endless playability, stunning artwork and fant...",0.017984,0.001343,-0.003260,0.032234,0.002442,-0.021048,0.008277,-0.000148,...,0.009077,-0.019254,-0.009203,0.005457,-0.005860,0.011202,-0.004034,-0.017642,-0.040061,-0.009398
3,1642293087,"In 'Coral', use dice to make shapes in 3D! In ...",0.035804,-0.011564,-0.010287,0.004304,-0.009367,0.016175,0.008818,-0.045228,...,-0.015247,0.024286,-0.059678,0.042627,0.017888,0.007787,-0.003178,-0.007208,-0.027904,-0.051965
4,932587626,Online 3D Stores for businesses that blur 3D G...,0.085045,-0.055648,-0.040125,0.035144,0.014326,-0.000312,-0.034597,-0.014971,...,-0.000892,-0.057838,0.042626,-0.003969,0.002463,0.042471,0.053249,0.005034,-0.009061,0.000517


In [26]:
def Recommender(Hp, query2, topic_blurb, top_n=10):
    
    cols = ['topic_0', 'topic_1', 'topic_2', 'topic_3',
       'topic_4', 'topic_5', 'topic_6', 'topic_7', 'topic_8', 'topic_9',
       'topic_10', 'topic_11', 'topic_12', 'topic_13', 'topic_14', 'topic_15',
       'topic_16', 'topic_17', 'topic_18', 'topic_19']
    
    query = pre_process2(query2)
    # Transform the query using the same vectorizer as above
    doc_q2 = vectorizer.transform([query2])
    # Use lsa model from above to transform the vectorized query
    doc_topic_q2 = lsa.transform(doc_q2)

    # Create a dataframe of query2
    
    Hp = Hp[cols]
    Qp = pd.DataFrame(doc_topic_q2.round(3),
                    index= [topic_blurb],
                    columns = cols)

    # cosine similarities
    cos_sim = similarEntries(Hp, Qp)

    # Add a new column
    Hp['similarity']= cos_sim
    return Hp, Qp

# make a function that outputs another dataframe with a 'similarity' column added
def similarEntries(Hp, Qp,n=3):
    '''function returns another dataframe
    '''
    nrows = Hp.shape[0]
    Hp = Hp.iloc[:,:-1] ## <- without title
    Hp = np.asarray(Hp)
    Qp = np.asarray(Qp)
    
    out = []
    for j in range(nrows):
        out.append(cosine_similarity(Qp.reshape(1,-1), Hp[j,:].reshape(1,-1)))
    
    cos_sim = out
    
    return [each[0][0] for each in cos_sim]

In [21]:
query2 = 'Endless playability, stunning artwork and fantasy'

In [23]:
import re

In [29]:
nlp = spacy.load('en', disable=['parser', 'ner'])

In [32]:
Recommender(df_lsa_model_20, query2, 'topic_blurb', top_n=10)

NotFittedError: TfidfVectorizer - Vocabulary wasn't fitted.